## 0. 모듈 불러오기

In [1]:
import os
import glob
from pathlib import Path

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random
import math

import pickle
import cv2
import mediapipe as mp

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 1. 데이터 확인

In [4]:
#한 폴더당 중심 동작 20개씩 뽑음
select_df=pd.read_csv('C:/Users/HOME/Desktop/AI_project/KSL/DF_select.csv')
select_df

,name,fold,class,num,path,label
0,00_01.0045.jpg,00_01,1,45,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
1,00_01.0046.jpg,00_01,1,46,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
2,00_01.0047.jpg,00_01,1,47,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
3,00_01.0048.jpg,00_01,1,48,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
4,00_01.0049.jpg,00_01,1,49,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/0...,hi
...,...,...,...,...,...,...
3595,19_77.0072.jpg,19_77,77,72,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3596,19_77.0073.jpg,19_77,77,73,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3597,19_77.0074.jpg,19_77,77,74,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care
3598,19_77.0075.jpg,19_77,77,75,C:/Users/HOME/Desktop/AI_project/KSL/KSL_rgb/1...,care


In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.25)
train_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'training')

valid_generator = datagen.flow_from_dataframe(
          dataframe=select_df,
          batch_size=128,
          class_mode='categorical',
          shuffle=True,
          x_col='path',
          y_col='label',
          subset = 'validation')

### CNN

In [ ]:
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow import keras

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
pool_size = 2
    
model1 = Sequential([
  keras.Input(shape=(256,256,3)),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),                                           
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 50, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 80, kernel_size = (3,3), padding='same'),  
  BatchNormalization(),                                       
  Activation('relu'),   
  MaxPooling2D(pool_size=pool_size),                         

  Flatten(),
  Dense(50),
  Activation('relu'),                                          
  
  Dense(10, activation='softmax'),
])

model1.summary()

In [ ]:
model1.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
#cnn
history1= model1.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)

### CNN+LSTM

In [ ]:
from keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, MaxPooling2D,\
                        Permute, TimeDistributed, Bidirectional,GRU, LeakyReLU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Reshape
from keras import models
from keras import layers
import tensorflow as tf

In [ ]:
Model = Sequential()

Model.add(SeparableConv2D(12,(3,3),activation="relu",
                 input_shape=(256,256,3)))
Model.add(BatchNormalization())
Model.add(MaxPooling2D((2,2)))

#
Model.add(SeparableConv2D(24,(3,3),
                 activation="relu",padding="same"))
Model.add(Dropout(0.2))
Model.add(MaxPooling2D((2,2)))

#
Model.add(TimeDistributed(Flatten()))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))
Model.add(Bidirectional(LSTM(64,
                                  return_sequences=True,
                                  dropout=0.5,
                                  recurrent_dropout=0.5)))

#
Model.add(Flatten())
Model.add(Dense(128,activation="relu"))
Model.add(Dropout(0.5))
Model.add(Dense(10,activation="sigmoid"))

In [ ]:
Model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
checkpoint_filepath = 'C:/Users/HOME/Desktop/AI_project/KSL/h5/' + 'best_model_{val_accuracy:.4f}.h5'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [ ]:
history2= Model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)

### VGG16
- 하려다가 실패함

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3), classes=10)

In [ ]:
base_model.trainable = False

In [ ]:
base_model.compile(
  optimizer='adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

In [ ]:
history3= base_model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator,
    shuffle=True)